# Using Napari to interactively visualize cell segmentation

For a more complete version of this example, with text describing each step, please refer to the [skimage tutorials](https://github.com/scikit-image/skimage-tutorials/).

In [ ]:
%matplotlib inline
%gui qt5

from matplotlib import cm
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection, Line3DCollection

import numpy as np

from scipy import ndimage as ndi
from scipy import stats

from skimage import (exposure, feature, filters, io, measure,
                      morphology, restoration, segmentation, transform,
                      util)

The cell data is a 60MB file, so it may take a while for the following to complete.

In [ ]:
data = io.imread("https://github.com/scikit-image/skimage-tutorials/raw/master/images/cells.tif")

print("shape: {}".format(data.shape))
print("dtype: {}".format(data.dtype))
print("range: ({}, {})".format(data.min(), data.max()))

We don't use the spacing information calculated below, but it's good to have in case you want to apply, e.g., a Gaussian filter:

In [ ]:
# The microscope reports the following spacing
original_spacing = np.array([0.2900000, 0.0650000, 0.0650000])

# We downsampled each slice 4x to make the data smaller
rescaled_spacing = original_spacing * [1, 4, 4]

# Normalize the spacing so that pixels are a distance of 1 apart
spacing = rescaled_spacing / rescaled_spacing[2]

print("microscope spacing: {}\n".format(original_spacing))
print("after rescaling images: {}\n".format(rescaled_spacing))
print("normalized spacing: {}\n".format(spacing))

After the viewer pops up, play around with the sliders at the bottom and left.

In [ ]:
import napari
view = napari.view(data)

## Exposure

In [ ]:
vmin, vmax = stats.scoreatpercentile(data, (0.5, 99.5))

rescaled = exposure.rescale_intensity(
    data, 
    in_range=(vmin, vmax), 
    out_range=np.float32
).astype(np.float32)

In [ ]:
view.add_image(rescaled, name='Rescaled');

## Edge detection

In [ ]:
sobel = np.zeros_like(rescaled)
for plane, image in enumerate(rescaled):
    sobel[plane] = filters.sobel(image)

In [ ]:
view.add_image(sobel, name='Sobel');

## Filters

In [ ]:
rescaled_uint8 = util.img_as_ubyte(rescaled)

denoised = np.zeros_like(rescaled_uint8)

for plane, image in enumerate(rescaled_uint8):
    denoised[plane] = filters.median(image)
    
view.add_image(denoised, name='Median');

## Thresholding

In [ ]:
threshold_li = filters.threshold_li(denoised)
li = denoised >= threshold_li

threshold_otsu = filters.threshold_otsu(denoised)
otsu = denoised >= threshold_otsu

view.add_image(otsu, name='Thresholded');

## Morphological operations

In [ ]:
width = 20

remove_holes = morphology.remove_small_holes(
    otsu, 
    min_size=width ** 3
)

In [ ]:
width = 20

remove_objects = morphology.remove_small_objects(
    remove_holes, 
    min_size=width ** 3
)

In [ ]:
view.add_image(remove_objects, name='Fill and remove');

## Segmentation

In [ ]:
labels = measure.label(remove_objects)

In [ ]:
view.add_labels(labels + 1, name='Initial labeling');

In [ ]:
distance = ndi.distance_transform_edt(remove_objects)

In [ ]:
view.add_image(distance, name='Distance map', colormap='viridis');

In [ ]:
peak_local_max = feature.peak_local_max(
    distance,
    footprint=np.ones((15, 15, 15), dtype=np.bool),
    indices=False,
    labels=measure.label(remove_objects)
)

markers = measure.label(peak_local_max)

In [ ]:
view.add_points(np.array(np.nonzero(markers)).T,
                n_dimensional=True);

In [ ]:
labels = morphology.watershed(
    -distance,
    markers,
    mask=remove_objects
)

In [ ]:
view.add_labels(labels + 1, name='Watershed', opacity=1);

The following segmentation isn't all that great (we started with too many markers inside a single cell).  To improve the situation, we first blur the distance map, then repeat the whole procedure:

In [ ]:
distance_smooth = filters.gaussian(distance, sigma=10)
features = feature.peak_local_max(
    distance_smooth
)
markers = measure.label(features)

labels = morphology.watershed(
    -distance_smooth,
    mask=remove_objects
)

In [ ]:
view.add_labels(labels + 1, name='Watershed (smoothed)', opacity=1);